## Generate Static Graphs

These are the input parameters for the notebook. They will be automatically changed when the scripts to generate monthly statistics are run. You can modify them manually to generate multiple plots locally as well.

Pass in `None` to remove the filters and plot all data. This is not recommended for production settings, but might be useful for reports based on data snapshots.

In [ ]:
year = 2020
month = 11
program = "default"
study_type = "study"
mode_of_interest = None

In [ ]:
from collections import defaultdict

import numpy as np
import pandas as pd

from plots import *
import scaffolding

sns.set_style("whitegrid")
sns.set()
%matplotlib inline

In [ ]:
# Loading mapping dictionaries from mapping_dictionaries notebook
%store -r dic_re
%store -r dic_pur

# convert a dictionary to a defaultdict
dic_re = defaultdict(lambda: 'Other',dic_re)
dic_pur = defaultdict(lambda: 'Other',dic_pur)

## Collect Data From Database

In [ ]:
expanded_ct, file_suffix, quality_text = scaffolding.load_viz_notebook_data(year,
                                                                            month,
                                                                            program,
                                                                            study_type,
                                                                            dic_re,
                                                                            dic_pur=dic_pur)

## Generic Metrics

### Distribution of Mode_confirm attribute

In [ ]:
labels_mc = expanded_ct['Mode_confirm'].value_counts(dropna=True).keys().tolist()
values_mc = expanded_ct['Mode_confirm'].value_counts(dropna=True).tolist()
plot_title= "Number of trips for each mode (selected by users)\n%s" % quality_text
file_name= 'ntrips_mode_confirm%s' % file_suffix
pie_chart_mode(plot_title,labels_mc,values_mc,file_name)
alt_text = store_alt_text_pie(pd.DataFrame(values_mc, labels_mc), file_name, plot_title)
print(expanded_ct['Mode_confirm'].value_counts(dropna=True))

In [ ]:
labels_mc = expanded_ct.query("Trip_purpose == 'Work'").Mode_confirm.value_counts(dropna=True).keys().tolist()
values_mc = expanded_ct.query("Trip_purpose == 'Work'").Mode_confirm.value_counts(dropna=True).tolist()
plot_title= "Number of commute trips for each mode (selected by users)\n%s" % quality_text
file_name= 'ntrips_commute_mode_confirm%s' % file_suffix
pie_chart_mode(plot_title,labels_mc,values_mc,file_name)
alt_text = store_alt_text_pie(pd.DataFrame(values_mc, labels_mc), file_name, plot_title)
print(expanded_ct.query("Trip_purpose == 'Work'").Mode_confirm.value_counts(dropna=True))

### Distribution of Trip_purpose attribute

In [ ]:
labels_tp = expanded_ct['Trip_purpose'].value_counts(dropna=True).keys().tolist()
values_tp = expanded_ct['Trip_purpose'].value_counts(dropna=True).tolist()
plot_title="Number of trips for each purposes (selected by users)\n%s" % quality_text
file_name= 'ntrips_purpose%s' % file_suffix
pie_chart_purpose(plot_title,labels_tp,values_tp,file_name)
alt_text = store_alt_text_pie(pd.DataFrame(values_tp, labels_tp), file_name, plot_title)
print(expanded_ct['Trip_purpose'].value_counts(dropna=True))

### Mode choice for trips under 10 miles

In [ ]:
labels_d10 = expanded_ct.loc[(expanded_ct['distance_miles'] <= 10)].Mode_confirm.value_counts(dropna=True).keys().tolist()
values_d10 = expanded_ct.loc[(expanded_ct['distance_miles'] <= 10)].Mode_confirm.value_counts(dropna=True).tolist()
plot_title="Mode confirmations for trips under 10 Miles\n%s" % quality_text
file_name ='ntrips_under10miles_mode_confirm%s' % file_suffix
pie_chart_mode(plot_title,labels_d10,values_d10,file_name)
alt_text = store_alt_text_pie(pd.DataFrame(values_d10, labels_d10), file_name, plot_title)
print(expanded_ct.loc[(expanded_ct['distance_miles'] <= 10)].Mode_confirm.value_counts(dropna=True))

### Miles per chosen transport mode

In [ ]:
miles = expanded_ct.groupby('Mode_confirm').agg({'distance_miles': ['sum', 'count' , 'mean']})
miles.columns = ['Total (miles)', 'Count', 'Average (miles)']
miles = miles.reset_index()
miles =miles.sort_values(by=['Total (miles)'], ascending=False)

#data
miles_dict = dict(zip(miles['Mode_confirm'], miles['Total (miles)']))

labels_m = []
values_m = []

for x, y in miles_dict.items():
    labels_m.append(x)
    values_m.append(y)

plot_title="Miles for each mode (selected by users)\n%s" % quality_text
file_name ='miles_mode_confirm%s' % file_suffix
pie_chart_mode(plot_title,labels_m,values_m,file_name)
alt_text = store_alt_text_pie(pd.DataFrame(values_m, labels_m), file_name, plot_title)
print(miles)

### Average miles per transport mode selected (Mode_confirm)

In [ ]:
data = miles.drop((miles.query("Count < 3").index)).sort_values(by=['Average (miles)'], ascending=False)
x='Mode_confirm'
y='Average (miles)'

plot_title=" Average Miles for each mode with > 3 entries\n%s" % quality_text
file_name ='average_miles_mode_confirm%s' % file_suffix

barplot_mode(data,x,y,plot_title,file_name)
alt_text = store_alt_text_bar(pd.DataFrame(data['Average (miles)'].values, data['Mode_confirm']), file_name, plot_title)

### Number of trips by day¶

In [ ]:
fq_days = expanded_ct.groupby(['start_local_dt_day']).agg({'start_local_dt_day': ['sum', 'count']})
fq_days = fq_days.reset_index()
fq_days.columns = ['Day of the Month', 'Total', 'Number of Trips']

data = fq_days
x = 'Day of the Month'
y = 'Number of Trips'

plot_title="Number of trips by day\n%s" % quality_text
file_name ='ntrips_per_day%s' % file_suffix

barplot_day(data,x,y,plot_title,file_name)
alt_text = store_alt_text_bar(pd.DataFrame(data['Number of Trips'].values, data['Day of the Month']), file_name, plot_title)

### Number of trips by day of week¶

In [ ]:
fq_weekdays = expanded_ct.groupby(['start_local_dt_weekday']).agg({'start_local_dt_weekday': ['sum', 'count']})
fq_weekdays = fq_weekdays.reset_index()
fq_weekdays.columns = ['Weekday', 'Total', 'Number of Trips']
weekday_labels = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]
fq_weekdays["Weekday"] = fq_weekdays.Weekday.apply(lambda x: weekday_labels[x])

data = fq_weekdays
x = 'Weekday'
y = 'Number of Trips'

plot_title="Number of trips by weekday\n%s" % quality_text
file_name ='ntrips_per_weekday%s' % file_suffix

barplot_day(data,x,y,plot_title,file_name)
alt_text = store_alt_text_bar(pd.DataFrame(data['Number of Trips'].values, data['Weekday']), file_name, plot_title)